In [1]:
from alphatools.research import run_pipeline, get_symbols
from alphatools.data import Factory
import pandas as pd
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.filters import StaticAssets

In [2]:
assets = get_symbols(['A', 'AAL'], as_of_date=None)

In [3]:
assets

[Equity(0 [A]), Equity(2 [AAL])]

In [4]:
p = Pipeline(
    columns={
        'col_A': Factory['sample'].value.latest,
    },
    screen=StaticAssets(assets)
)

In [5]:
df = run_pipeline(
    p,
    pd.Timestamp('2016-01-05', tz='utc'),
    pd.Timestamp('2018-01-04', tz='utc')
)

In [6]:
df.head()

col_A
2016-01-05 00:00:00+00:00 Equity(0 [A])    0.543405
                          Equity(2 [AAL])  0.424518
2016-01-06 00:00:00+00:00 Equity(0 [A])    0.543405
                          Equity(2 [AAL])  0.424518
2016-01-07 00:00:00+00:00 Equity(0 [A])    0.543405

In [12]:
import re, os


In [36]:

os.environ["USERN"] = "jonathan"
os.environ["PASS"] = "password1"

In [37]:
text = "$USERN:$PASS:something else"

In [38]:
os.path.expandvars(text)

'jonathan:password1:something else'